In [1]:
import sys
import json
import csv
import yaml

import importlib

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
sys.path.append('/Users/Nfaith21/ECS 116/')
import util_2 as util

In [2]:
from pymongo import MongoClient

client = MongoClient()
# could have written client = MongoClient("localhost", 27017)
#                 or client = MongoClient("mongodb://localhost:27017/")

In [3]:
# I have (or will have) a database "airbnb"
db = client.airbnb


print('The list of all databases currently in the MongoDB client is:')
print(client.list_database_names())

print('\nThe list of all collections in the airbnb database is:')
print(db.list_collection_names())

The list of all databases currently in the MongoDB client is:
['admin', 'airbnb', 'company', 'config', 'local', 'test']

The list of all collections in the airbnb database is:
['newlistings', 'listings', 'reviews', 'calendar', 'listings_with_reviews_m', 'listings_with_calendar']


### MongoDB Pipeline to Create listings_with_reviews_and_cal

In [11]:
db.listings_with_reviews_and_cal.drop()


pipeline = [
    {"$lookup":
       {
         "from": "listings_with_calendar",
         "localField": "id",
         "foreignField": "_id",
         "as": "cal_docs"
       }}, 
    
    { "$unwind": { "path": "$cal_docs",
"preserveNullAndEmptyArrays" : True}},
    
    {"$addFields": {
      "first_available_date": "$$ROOT.cal_docs.first_available_date",
      "last_available_date": "$$ROOT.cal_docs.last_available_date",
        "average_price": "$$ROOT.cal_docs.average_price",
        "dates_list": "$$ROOT.cal_docs.dates_list"
    }},
    { "$unset": "cal_docs" },
    
    { "$out": "listings_with_reviews_and_cal" }
    
]

time1 = datetime.now()
test1 = db.listings_with_reviews_m.aggregate(pipeline)
time2 = datetime.now()
diff = util.time_diff(time1, time2)

print('\nTime it took was:', format(diff, '.4f'), '.')

print(db.list_collection_names())

print("test1:")

# print(len(list(test1)))

print(type(test1))




Time it took was: 23.7086 .
['newlistings', 'listings', 'listings_with_reviews_and_cal', 'reviews', 'calendar', 'listings_with_reviews_m', 'listings_with_calendar']
test1:
<class 'pymongo.command_cursor.CommandCursor'>


In [16]:
count = db.listings_with_reviews_and_cal.count_documents({})
print(count)

39202


In [20]:
doc = db.listings_with_reviews_and_cal.find_one()
pprint.pp(doc)

{'_id': ObjectId('6661886f7a796e380285229e'),
 'id': '977395984065981849',
 'name': 'Home in Brooklyn · 1 bedroom · 1 bed · 1 bath',
 'host_id': '95344065',
 'host_name': 'Derek',
 'neighbourhood_group': 'Brooklyn',
 'neighbourhood': 'Sheepshead Bay',
 'latitude': 40.59179,
 'longitude': -73.94285,
 'room_type': 'Private room',
 'price': 30.0,
 'minimum_nights': 31,
 'number_of_reviews': 1,
 'last_review': datetime.datetime(2024, 1, 3, 0, 0),
 'reviews_per_month': 0.86,
 'calculated_host_listings_count': 7,
 'availability_365': 339,
 'number_of_reviews_ltm': 1,
 'license': '',
 'reviews': [{'_id': ObjectId('666188737a796e380294b8ce'),
              'listing_id': '977395984065981849',
              'id': '1060927930986644037',
              'date': datetime.datetime(2024, 1, 3, 0, 0),
              'reviewer_id': '56179331',
              'reviewer_name': 'Sean',
              'comments': 'I fear it’s kind of a “you get what you pay for” '
                          'situation. The place

### JSON File Loading

In [21]:
def convert_lwc_to_json(doc):
    doc_new = {}
    doc_new['_id'] = str(doc['_id'])
    for key in ['id', 'name', 'host_id', 'host_name', 'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews']:
        doc_new[key] = doc[key] if not pd.isna(doc[key]) else None
    doc_new['last_review'] = doc['last_review'].strftime('%Y-%m-%d') if not pd.isna(doc['last_review']) else None
    for key in ['reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'number_of_reviews_ltm', 'license']:
        doc_new[key] = doc[key] if not pd.isna(doc[key]) else None
        
    doc_new['average_price'] = doc['average_price']
    for key in ['first_available_date', 'last_available_date']:
        doc_new[key] = doc[key].strftime('%Y-%m-%d')
        
    dlist = []
    for d in doc['reviews']:
        d_new = {}
        d_new['_id'] = str(d['_id'])
        d_new['date'] = d['date'].strftime('%Y-%m-%d') if not pd.isna(d['date']) else None
        for key in ['listing_id', 'id', 'reviewer_id', 'reviewer_name', 'comments']:
            d_new[key] = d[key] if not pd.isna(d[key]) else None
        dlist.append(d_new)
    doc_new['reviews'] = dlist

    dlist = []
    for d in doc['dates_list']:
        d_new = {}
        d_new['date'] = d['date'].strftime('%Y-%m-%d')
        for key in ['price', 'minimum_nights', 'maximum_nights', 'available']:
            d_new[key] = d[key]
        dlist.append(d_new)
    doc_new['dates_list'] = dlist
    
    return doc_new

pprint.pp(convert_lwc_to_json(doc))

{'_id': '6661886f7a796e380285229e',
 'id': '977395984065981849',
 'name': 'Home in Brooklyn · 1 bedroom · 1 bed · 1 bath',
 'host_id': '95344065',
 'host_name': 'Derek',
 'neighbourhood_group': 'Brooklyn',
 'neighbourhood': 'Sheepshead Bay',
 'latitude': 40.59179,
 'longitude': -73.94285,
 'room_type': 'Private room',
 'price': 30.0,
 'minimum_nights': 31,
 'number_of_reviews': 1,
 'last_review': '2024-01-03',
 'reviews_per_month': 0.86,
 'calculated_host_listings_count': 7,
 'availability_365': 339,
 'number_of_reviews_ltm': 1,
 'license': '',
 'average_price': 30.0,
 'first_available_date': '2024-02-06',
 'last_available_date': '2025-02-04',
 'reviews': [{'_id': '666188737a796e380294b8ce',
              'date': '2024-01-03',
              'listing_id': '977395984065981849',
              'id': '1060927930986644037',
              'reviewer_id': '56179331',
              'reviewer_name': 'Sean',
              'comments': 'I fear it’s kind of a “you get what you pay for” '
            

In [22]:
print(db.listings_with_reviews_and_cal.count_documents({}))

cursor = db.listings_with_reviews_and_cal.find({'id' : {'$regex' : '^1000.*$'}})
    
l = list(cursor)
print(len(l))

39202
43


In [23]:
output = []

for doc in l:
    output.append(convert_lwc_to_json(doc))

print(len(output))

43


In [24]:
# Writing dict to a json file into a json file in a subdirectory
# Also putting this function into my util.py
def write_dict_to_dir_json(dict, dir, filename):
    with open(dir + '/' + filename, 'w') as fp:
        json.dump(dict, fp)

dir = '/Users/Nfaith21/ECS 116'
filename = 'listings_with_reviews_and_cal_subset_1000.json'
write_dict_to_dir_json(output, dir, filename)

